In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-22 18:13:05--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-22 18:13:05 (14.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [28]:
import minsearch

In [29]:
import json 

In [30]:
with open('documents.json','r') as f_in:
    docs_raw = json.load(f_in)

In [31]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']= course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [32]:
index = minsearch.Index(text_fields=["question", "text", "section"],
    keyword_fields=["course"])

In [33]:
q = 'the course has already stared, can I still enroll?'

In [34]:
index.fit(documents)

In [49]:
boost = {'question':3.0, 'section':0.5}

results = index.search(query= q,
             boost_dict=boost)

In [45]:
from openai import OpenAI
from os import getenv

# gets API Key from environment variable OPENAI_API_KEY
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

response = client.chat.completions.create(
  model="openai/gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": q,
    },
  ],
)


In [47]:
response.choices[0].message.content

"It depends on the policies of the institution or organization offering the course. Some may allow late enrollment with special permission or a fee, while others may have a strict deadline for enrollment. It's advisable to check with the course administrator or registrar's office to see if it's still possible to enroll in the course."

In [86]:
prompt_template= """ 
You're a course teaching assistant. 
Answer the question based on the context. 
use only the facts in the contexts for answering the question. if the context doesn't contain the answer output should be I don't know.


Question: {question}

CONTEXT":{context}
""".strip()

In [79]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"


In [88]:
prompt = prompt_template.format(question=q, context=context).strip()

In [89]:
print(prompt)

You're a course teaching assistant. 
Answer the question based on the context. 
use only the facts in the contexts for answering the question. if the context doesn't contain the answer output should be I don't know.


Question: the course has already stared, can I still enroll?

CONTEXT":section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that t

In [90]:
response = client.chat.completions.create(
  model="openai/gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": prompt,
    },
  ],
)


In [91]:
response.choices[0].message.content

'Yes, you can still enroll in the course even though it has already started.'